In [2]:
%config IPCompleter.greedy=True
from notebook.services.config import ConfigManager
c = ConfigManager()
c.update('notebook', {"CodeCell": {"cm_config": {"autoCloseBrackets": False}}})

{'CodeCell': {'cm_config': {'autoCloseBrackets': False}}}

In [3]:
from plnn.mnist_basic import Net
import torch
import torch.nn as nn
from torchvision.datasets.mnist import MNIST
import torch.utils.data
import numpy as np
import torch.nn.functional as F
from torchvision import datasets, transforms

In [4]:
def generate_domain(input_tensor, eps_size):
    return torch.stack((input_tensor - eps_size, input_tensor + eps_size))

In [5]:
model = Net()
model.load_state_dict(torch.load('save/mnist_cnn.pt'))
model.cuda()
dataset = MNIST('./data', train=True, download=True,
                transform=transforms.Compose([
                    transforms.ToTensor(),
                    transforms.Normalize((0.1307,), (0.3081,))
                ])),  # load the testing dataset
batch_size=10
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])),
    batch_size=batch_size, shuffle=False, pin_memory=True)
# test_loader = torch.utils.data.DataLoader(dataset, batch_size=1)#retrieve items 1 at a time
seed = 0

In [6]:
class VerificationNetwork(nn.Module):
    def __init__(self, in_features, out_features, base_network, out_function, true_class_index):
        super(VerificationNetwork, self).__init__()
        self.true_class_index = true_class_index
        self.out_function = out_function
        self.base_network = base_network
        self.out_features = out_features
        self.in_features = in_features
        self.property_layer = self.attach_property_layers(self.base_network, self.true_class_index)

    '''need to  repeat this method for each class so that it describes the distance between the corresponding class 
    and the closest other class'''
    def attach_property_layers(self, model: Net, true_class_index: int):
        n_classes = model.fc2.out_features
        cases = []
        for i in range(n_classes):
            if i == true_class_index:
                continue
            case = [0] * n_classes  # list of zeroes
            case[true_class_index] = 1  # sets the property to 1
            case[i] = -1
            cases.append(case)
        weights = np.array(cases)
        weightTensor = nn.Linear(in_features=n_classes, out_features=n_classes,
                                 bias=False)
        weightTensor.weight.data = torch.from_numpy(weights).float()
        return weightTensor

    def forward(self, x):
        x = self.base_network(x)
        x = self.property_layer(x)
        print(x)
        print(x.size())
        return torch.min(x,dim=1,keepdim=True)


In [7]:
#get the data and label
data, target =next(iter(test_loader))
print(f'data size:{data.size()}')
# print(data[0])
# create the domain
domain_raw = generate_domain(data,0.001)
data_size=data.size()
print(f'domain size:{domain_raw.size()}')


data size:torch.Size([10, 1, 28, 28])
domain size:torch.Size([2, 10, 1, 28, 28])


In [8]:
test_out=model(data.cuda())
print(test_out.size())

torch.Size([10, 10])


In [59]:
domain=domain_raw.view(2,batch_size,-1)
print(domain.size())

torch.Size([2, 10, 784])


In [185]:
# global_ub_point, global_ub = net.get_upper_bound(domain)
# global_lb = net.get_lower_bound(domain)


def get_upper_bound(domain):
    #we try get_upper_bound
    nb_samples = 1024
    nb_inp = domain.size()[2:]  #get last dimensions
    print(nb_inp)
    # Not a great way of sampling but this will be good enough
    # We want to get rows that are >= 0
    rand_samples_size = [batch_size, nb_samples] + list(nb_inp)
    print(rand_samples_size)
    rand_samples = torch.zeros(rand_samples_size)
    print(rand_samples.size())
    # print(rand_samples)
    rand_samples.uniform_(0, 1)
    # print(rand_samples)
    print(rand_samples.size())
    domain_lb = domain.select(0, 0).contiguous()
    domain_ub = domain.select(0, 1).contiguous()
    # print(domain_lb)
    print(domain_lb.size())
    # print(domain_ub)
    print(domain_ub.size())
    domain_width = domain_ub - domain_lb
    print(domain_width.size())
    print(domain_lb.view([batch_size, 1] + list(nb_inp)).size())
    print(domain_width.view([batch_size, 1] + list(nb_inp)).size())
    domain_lb = domain_lb.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the initial point for the number of examples
    print(domain_lb.size())
    domain_width = domain_width.view([batch_size, 1] + list(nb_inp)).expand(
        [batch_size, nb_samples] + list(nb_inp))  #expand the width for the number of examples
    print(domain_width.size())
    #those should be the same
    print(domain_width.size())
    print(rand_samples.size())
    inps = domain_lb + domain_width * rand_samples
    # print(inps) #each row shuld be different
    print(inps.size())
    #now flatten the first dimension into the second
    flattened_size = [inps.size(0) * inps.size(1)] + list(inps.size()[2:])
    print(flattened_size)
    #rearrange the tensor so that is consumable by the model
    print(data_size)
    examples_data_size = [flattened_size[0]] + list(data_size[1:])  #the expected dimension of the example tensor
    print(examples_data_size)
    var_inps = torch.Tensor(inps).view(examples_data_size)
    print(var_inps.size())  #should match data_size
    print(inps.size())
    # print(inps[0][0])
    # print(inps[0][1])
    # print(var_inps[0][0])
    # print(var_inps[1][0])
    outs = model.forward(var_inps.cuda())  #gets the input for the values
    print(outs.size())
    print(outs[0])  #those two should be very similar but different because they belong to two different random examples
    print(outs[1])
    print(target.unsqueeze(1))
    target_expanded = target.unsqueeze(1).expand(
        [batch_size, nb_samples])  #generates nb_samples copies of the target vector, all rows should be the same
    print(target_expanded.size())
    print(target_expanded)
    target_idxs = target_expanded.contiguous().view(
        batch_size * nb_samples)  #contains a list of indices that tells which columns out of the 10 classes to pick
    print(target_idxs.size())  #the first dimension should match
    print(outs.size())
    print(outs[target_idxs[0]].size())
    outs_true_class = outs.gather(1, target_idxs.cuda().view(-1,
                                                             1))  #we choose dimension 1 because it's the one we want to reduce
    print(outs_true_class.size())
    # print(outs[0])
    # print(target_idxs[1])
    # print(outs[1][0])#these two should be similar but different because they belong to different examples
    # print(outs[0][0])
    print(outs_true_class.size())
    outs_true_class_resized = outs_true_class.view(batch_size, nb_samples)
    print(outs_true_class_resized.size())  #resize outputs so that they each row is a different element of each batch
    upper_bound, idx = torch.min(outs_true_class_resized,
                                 dim=1)  #this returns the distance of the network output from the given class, it selects the class which is furthest from the current one
    print(upper_bound.size())
    print(idx.size())
    print(idx)
    print(upper_bound)
    # rearranged_idx=idx.view(list(inps.size()[0:2]))
    # print(rearranged_idx.size()) #rearranged idx contains the indexes of the minimum class for each example, for each element of the batch
    print(f'idx size {idx.size()}')
    print(f'inps size {inps.size()}')
    print(idx[0])
    # upper_bound = upper_bound[0]
    unsqueezed_idx = idx.cuda().view(-1, 1)
    print(f'single size {inps[0][unsqueezed_idx[0][0]][:].size()}')
    print(f'single size {inps[1][unsqueezed_idx[1][0]][:].size()}')
    print(f'single size {inps[2][unsqueezed_idx[2][0]][:].size()}')
    ub_point = [inps[x][idx[x]][:].numpy() for x in range(idx.size()[0])]
    ub_point = torch.tensor(ub_point)
    print(
        ub_point)  #ub_point represents the input that amongst all examples returns the minimum response for the appropriate class
    print(ub_point.size())
    # print(unsqueezed_idx.size())
    # ub_point = torch.gather(inps.cuda(),1,unsqueezed_idx.cuda())#todo for some reason it doesn't want to work
    # print(ub_point.size())
    return ub_point, upper_bound


In [186]:
#test the method
get_upper_bound(domain)

torch.Size([784])
[10, 1024, 784]
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 784])
torch.Size([10, 784])
torch.Size([10, 784])
torch.Size([10, 1, 784])
torch.Size([10, 1, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
torch.Size([10, 1024, 784])
[10240, 784]
torch.Size([10, 1, 28, 28])
[10240, 1, 28, 28]
torch.Size([10240, 1, 28, 28])
torch.Size([10, 1024, 784])
torch.Size([10240, 10])
tensor([-1.6408e+01, -1.5530e+01, -1.2936e+01, -9.0587e+00, -2.0639e+01,
        -1.8038e+01, -3.0886e+01, -1.2112e-04, -1.4305e+01, -1.3569e+01],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([-1.6407e+01, -1.5529e+01, -1.2936e+01, -9.0581e+00, -2.0639e+01,
        -1.8038e+01, -3.0886e+01, -1.2112e-04, -1.4304e+01, -1.3568e+01],
       device='cuda:0', grad_fn=<SelectBackward>)
tensor([[7],
        [2],
        [1],
        [0],
        [4],
        [1],
        [4],
        [9],
        [5],


tensor([[7, 7, 7,  ..., 7, 7, 7],
        [2, 2, 2,  ..., 2, 2, 2],
        [1, 1, 1,  ..., 1, 1, 1],
        ...,
        [9, 9, 9,  ..., 9, 9, 9],
        [5, 5, 5,  ..., 5, 5, 5],
        [9, 9, 9,  ..., 9, 9, 9]])
torch.Size([10240])
torch.Size([10240, 10])
torch.Size([10])
torch.Size([10240, 1])
torch.Size([10240, 1])
torch.Size([10, 1024])
torch.Size([10])
torch.Size([10])
tensor([  0,   0, 570,   0,  48, 724, 827, 624,  23,  96], device='cuda:0')
tensor([-1.2112e-04,  0.0000e+00, -3.4809e-04, -3.0518e-05, -1.7262e-04,
        -1.3924e-04, -3.4504e-03, -6.1989e-04, -4.0483e-03, -1.1921e-04],
       device='cuda:0', grad_fn=<MinBackward0>)
idx size torch.Size([10])
inps size torch.Size([10, 1024, 784])
tensor(0, device='cuda:0')
single size torch.Size([784])
single size torch.Size([784])
single size torch.Size([784])
tensor([[-0.4248, -0.4245, -0.4237,  ..., -0.4246, -0.4240, -0.4233],
        [-0.4237, -0.4250, -0.4249,  ..., -0.4247, -0.4239, -0.4235],
        [-0.4251, -0.4240,

(tensor([[-0.4248, -0.4245, -0.4237,  ..., -0.4246, -0.4240, -0.4233],
         [-0.4237, -0.4250, -0.4249,  ..., -0.4247, -0.4239, -0.4235],
         [-0.4251, -0.4240, -0.4234,  ..., -0.4233, -0.4243, -0.4238],
         ...,
         [-0.4248, -0.4237, -0.4237,  ..., -0.4246, -0.4235, -0.4233],
         [-0.4234, -0.4244, -0.4246,  ..., -0.4245, -0.4244, -0.4251],
         [-0.4245, -0.4237, -0.4239,  ..., -0.4249, -0.4236, -0.4249]]),
 tensor([-1.2112e-04,  0.0000e+00, -3.4809e-04, -3.0518e-05, -1.7262e-04,
         -1.3924e-04, -3.4504e-03, -6.1989e-04, -4.0483e-03, -1.1921e-04],
        device='cuda:0', grad_fn=<MinBackward0>))

In [188]:
#now try to do the lower bound
# import gurobipy as grb

ModuleNotFoundError: No module named 'gurobipy'

In [118]:
print(data.size())
true_class=torch.argmax(model(data.cuda()),dim=1)
print(true_class.size())
print(true_class)
single_true_class=true_class[0].item()
print(single_true_class)
verification_model=VerificationNetwork(28,10,model,model.forward,single_true_class)
verification_model.cuda()

result=verification_model.forward(data.cuda())
print(result)

torch.Size([10, 1, 28, 28])
torch.Size([10])
tensor([7, 2, 1, 0, 4, 1, 4, 9, 5, 9], device='cuda:0')
7


NameError: name 'VerificationNetwork' is not defined